In [19]:
#pip install textblob
#pip install spacy
#pip install afinn

### Sentiment Analysis
- The key aspect of sentiment analysis is to analyze a body of text for understanding the opinion expressed by it. Typically, we quantify this sentiment with a positive or negative value, called polarity. 
- The overall sentiment is often inferred as positive, neutral or negative from the sign of the polarity score.

### Method 1: VADER Sentiment Analysis:
- VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. It is fully open-sourced under the MIT License[http://choosealicense.com/] 

In [1]:
import spacy
from vaderSentiment import vaderSentiment

In [2]:
# Load the English language model from spaCy
english = spacy.load("en_core_web_sm")

In [3]:
analyzer = vaderSentiment.SentimentIntensityAnalyzer()

In [8]:
result = english("I was unhappy to see Trump speaking mostly in sentence")
sentences = [str(s) for s in result.sents]
sentiment = [analyzer.polarity_scores(str(s)) for s in sentences]

In [9]:
print(sentiment)

[{'neg': 0.231, 'neu': 0.661, 'pos': 0.107, 'compound': -0.3612}]


In [10]:
sentiment[0]['compound']

-0.3612

### Method 2: TextBlob Sentiment Analysis:
- TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [11]:
from textblob import TextBlob

In [12]:
text="I was unhappy to see Trump speaking mostly in sentence"
analysis =TextBlob(text)
print(analysis.sentiment.polarity)

-0.04999999999999999


### Method 3: AFINN Sentiment Analysis:
- AFINN is a list of words rated for valence with an integer between minus five (negative) and plus five (positive). This implementation uses AFINN-en-165.

In [36]:
# Initialize afinn sentiment analyzer
from afinn import Afinn
af = Afinn()

In [37]:
# Get af sentiment score
af.score("I was not happy to see Trump speaking mostly in sentence")

1.0

### Method 4: NLTK Corpus

In [38]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [39]:
lemmatizer = WordNetLemmatizer()

In [40]:
def penn_to_wn(tag):
    #Convert between the PennTreebank tags to simple Wordnet tags

    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [41]:
def sentiment_sentiwordnet(text):
    #text = text.decode("utf-8")
    raw_sentences = sent_tokenize(text)
    sentiment = 0
    tokens_count = 0

    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            word_sent = swn_synset.pos_score() - swn_synset.neg_score()

            if word_sent != 0:
                sentiment += word_sent
                tokens_count += 1

    if tokens_count == 0:
        return 0
    sentiment = sentiment/tokens_count
    if sentiment >= 0.01:
        return 1
    if sentiment <= -0.01:
        return -1
    return 0

In [42]:
sentiment_sentiwordnet("I was not happy to see Trump speaking mostly in sentences")

1